<a href="https://colab.research.google.com/github/marianoogimenez/google-asl-project/blob/master/PARTE_01_Recomendacion_de_contenido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import pandas as pd
import gensim
from nltk import SnowballStemmer

In [0]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'bucket_leo'
REGION = 'us-central1'

In [0]:
import tensorflow as tf
from google.cloud import bigquery
from google.colab import auth

### PARTE 01


In [5]:
auth.authenticate_user()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [6]:
client = bigquery.Client(project=PROJECT)
sql = """
SELECT * FROM AGEA_ASL.Dataset_A
"""
client.query(sql).to_dataframe().head()

,content_id,title,day,section_1,section_2,section_3,tag_1,tag_2,tag_3,tag_4,body
0,guHYkGEpg,Construyeron una casa ecológica que genera has...,2018-11-29,ARQ,-2,-2,casa,sustentabilidad,Arquitectura,-2,la sustentabilidad es el foco de la casa que p...
1,YzJkrHyK8,"Un reconocido urbanista, contra los autos autó...",2018-11-20,ARQ,-2,-2,Autos,-2,-2,-2,por john marcoff periodista de the new york ti...
2,jGo7Wm_-Q,Construyen un hotel 5 estrellas dentro de una ...,2018-12-04,ARQ,-2,-2,China,Shanghai,-2,-2,un hotel de lujo construido en el foso de una ...
3,fPdLtrf_q,Iban a inaugurar el mejor puente de Sudamérica...,2018-11-07,ARQ,-2,-2,Viste,Colombia,-2,-2,un nuevo error en una obra de ingeniería reabr...
4,MoqDKLXvL,"Cortinas de papel, una nueva tendencia que apo...",2018-10-20,ARQ,-2,-2,Diseño,Decoración,-2,-2,lleva años lleva años investigando sobre la ca...


In [0]:
df = client.query(sql).to_dataframe()

In [0]:
df.head()

,content_id,title,day,section_1,section_2,section_3,tag_1,tag_2,tag_3,tag_4,body
0,guHYkGEpg,Construyeron una casa ecológica que genera has...,2018-11-29,ARQ,-2,-2,casa,sustentabilidad,Arquitectura,-2,la sustentabilidad es el foco de la casa que p...
1,YzJkrHyK8,"Un reconocido urbanista, contra los autos autó...",2018-11-20,ARQ,-2,-2,Autos,-2,-2,-2,por john marcoff periodista de the new york ti...
2,jGo7Wm_-Q,Construyen un hotel 5 estrellas dentro de una ...,2018-12-04,ARQ,-2,-2,China,Shanghai,-2,-2,un hotel de lujo construido en el foso de una ...
3,fPdLtrf_q,Iban a inaugurar el mejor puente de Sudamérica...,2018-11-07,ARQ,-2,-2,Viste,Colombia,-2,-2,un nuevo error en una obra de ingeniería reabr...
4,MoqDKLXvL,"Cortinas de papel, una nueva tendencia que apo...",2018-10-20,ARQ,-2,-2,Diseño,Decoración,-2,-2,lleva años lleva años investigando sobre la ca...


In [0]:
# Parto del dataset A. Limpio las notas, las tokenizo y les saco las stopwords

def limpiador_cuerpo(x):
    x= x.strip()
    x= x.lower()
    x= re.sub("&#xe1;","á",x)
    x= re.sub("&#xe9;","é",x)
    x= re.sub("&#xed;","í",x)
    x= re.sub("&#xf3;","ó",x)
    x= re.sub("&#xfa;","ú",x)
    x= re.sub('<.*?>', '',x)
    x= re.sub(pattern="&quot;",repl="",string=x)
    x= re.sub("&.*?;"," ",x)
    x= re.sub(pattern="[^a-zA-Záéíóúñ \n]",repl="",string=x)
    return x

def sacar_stop(tokens,idioma="spanish",min_len = 3,stemizar= False):
    stopwords = nltk.corpus.stopwords.words(idioma)
    tok_filt = [token for token in tokens if token not in stopwords and len(token)>=min_len]
    #tok_filt = [token for token in tok_filt if token not in nltk.corpus.stopwords.words("english")]
    if stemizar:
        tok_filt = [self.stem.stem(token) for token in tok_filt]
    return tok_filt

In [0]:
notas_tokenizadas = [sacar_stop(limpiador_cuerpo(x).split(" ")) for x in df.body_raw]

In [0]:
# Vectorizo cada nota usando d2v

d2v = gensim.models.doc2vec.Doc2Vec.load("C:\\Users\\ldalera.AGEA\\Documents\\Google\\Recomendacion\\Codigos\\d2v.model") ## levanto el doc2vec entrenado con casi medio palo de notas


In [0]:
ans = []
for nota in notas_tokenizadas:
    ans.append(d2v.infer_vector(nota))
notas_vec = np.array(ans)
np.savetxt(X=notas_vec,fname="notas_datasetA_vectoriz.txt")

In [0]:
Borrar = pd.DataFrame(notas_vec)
borrar.head()